<a href="https://colab.research.google.com/github/deklund76/project-one/blob/Sarah/S3_Postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [91]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2021-11-07 22:36:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.63MB/s    in 0.2s    

2021-11-07 22:36:43 (5.63 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [92]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [103]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark.sql import SparkSession
url ="https://initial-datasets.s3.amazonaws.com/cases_deaths_norm.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("cases_deaths_norm.csv"), sep=",", header=True, inferSchema=True)
df.show()
df.columns

+---------------+---------+----+--------+--------+-----------+---------------------+----------------+---------+--------------+------------+----------------+-----------+----------+---------------+
|submission_date|mmwr_week|year|location|  concat|total_cases|total_confirmed_cases|total_prob_cases|new_cases|prob_new_cases|total_deaths|confirmed_deaths|prob_deaths|new_deaths|prob_new_deaths|
+---------------+---------+----+--------+--------+-----------+---------------------+----------------+---------+--------------+------------+----------------+-----------+----------+---------------+
|     2020-12-19|       51|2020|      AK|202051AK|      42659|                 null|            null|      325|             0|         267|            null|       null|         1|              0|
|     2020-12-26|       52|2020|      AK|202052AK|      44394|                 null|            null|      188|             0|         271|            null|       null|         1|              0|
|     2021-01-02|   

<bound method DataFrame.printSchema of DataFrame[submission_date: string, mmwr_week: int, year: int, location: string, concat: string, total_cases: int, total_confirmed_cases: int, total_prob_cases: int, new_cases: int, prob_new_cases: int, total_deaths: int, confirmed_deaths: int, prob_deaths: int, new_deaths: int, prob_new_deaths: int]>

In [104]:
url ="https://initial-datasets.s3.amazonaws.com/vaccinations_norm.csv"
spark.sparkContext.addFile(url)
df_2 = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vaccinations_norm.csv"), sep=",", header=True, inferSchema=True)
df_2.show()
df_2.columns

+----------+---------+----+--------+--------+-----------+------------+------------------+------------------+---------------+----------------+
|      Date|MMWR_week|Year|Location|  Concat|Distributed|Administered|Recip_Administered|Administered_Dose1|Series_Complete|Additional_Doses|
+----------+---------+----+--------+--------+-----------+------------+------------------+------------------+---------------+----------------+
|2020-12-19|       51|2020|      AK|202051AK|      26325|        1607|                 0|                 0|              0|            null|
|2020-12-26|       52|2020|      AK|202052AK|      45250|       11427|                 0|                 0|              0|            null|
|2021-01-02|       53|2021|      AK|202153AK|      54975|       18401|                 0|                 0|              0|            null|
|2021-01-09|        1|2021|      AK| 20211AK|      92875|       28539|                 0|                 0|              0|            null|
|2021-

<bound method DataFrame.printSchema of DataFrame[Date: string, MMWR_week: int, Year: int, Location: string, Concat: string, Distributed: int, Administered: int, Recip_Administered: int, Administered_Dose1: int, Series_Complete: int, Additional_Doses: int]>

In [106]:
cases_deaths_df = df.select([to_date("submission_date", "yyyy-MM-dd").alias("submission_date"),"mmwr_week","year","location","concat","total_cases","total_confirmed_cases","total_prob_cases","new_cases","prob_new_cases","total_deaths","confirmed_deaths","prob_deaths","new_deaths","prob_new_deaths"])
cases_deaths_df.show()
cases_deaths_df.columns

+---------------+---------+----+--------+--------+-----------+---------------------+----------------+---------+--------------+------------+----------------+-----------+----------+---------------+
|submission_date|mmwr_week|year|location|  concat|total_cases|total_confirmed_cases|total_prob_cases|new_cases|prob_new_cases|total_deaths|confirmed_deaths|prob_deaths|new_deaths|prob_new_deaths|
+---------------+---------+----+--------+--------+-----------+---------------------+----------------+---------+--------------+------------+----------------+-----------+----------+---------------+
|     2020-12-19|       51|2020|      AK|202051AK|      42659|                 null|            null|      325|             0|         267|            null|       null|         1|              0|
|     2020-12-26|       52|2020|      AK|202052AK|      44394|                 null|            null|      188|             0|         271|            null|       null|         1|              0|
|     2021-01-02|   

['submission_date',
 'mmwr_week',
 'year',
 'location',
 'concat',
 'total_cases',
 'total_confirmed_cases',
 'total_prob_cases',
 'new_cases',
 'prob_new_cases',
 'total_deaths',
 'confirmed_deaths',
 'prob_deaths',
 'new_deaths',
 'prob_new_deaths']

In [107]:
vaccines_df = df_2.select([to_date("Date", 'yyyy-MM-dd').alias("Date"),"MMWR_week","Year","Location","Concat","Distributed","Administered","Recip_Administered","Administered_Dose1","Series_Complete","Additional_Doses"])
vaccines_df.show()

+----------+---------+----+--------+--------+-----------+------------+------------------+------------------+---------------+----------------+
|      Date|MMWR_week|Year|Location|  Concat|Distributed|Administered|Recip_Administered|Administered_Dose1|Series_Complete|Additional_Doses|
+----------+---------+----+--------+--------+-----------+------------+------------------+------------------+---------------+----------------+
|2020-12-19|       51|2020|      AK|202051AK|      26325|        1607|                 0|                 0|              0|            null|
|2020-12-26|       52|2020|      AK|202052AK|      45250|       11427|                 0|                 0|              0|            null|
|2021-01-02|       53|2021|      AK|202153AK|      54975|       18401|                 0|                 0|              0|            null|
|2021-01-09|        1|2021|      AK| 20211AK|      92875|       28539|                 0|                 0|              0|            null|
|2021-

In [108]:
# Store environmental variable
# from getpass import getpass
# password = getpass('project-one')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cin6h4edfcqj.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root",
          "password": "project-one",
          "driver":"org.postgresql.Driver"}

In [110]:
# Write cases_deaths_df to casesdeaths table in RDS
cases_deaths_df.write.jdbc(url=jdbc_url, table = 'casesdeaths', mode = mode,properties = config)


In [112]:
# Write vaccines_df to vaccinations table in RDS
vaccines_df.write.jdbc(url=jdbc_url,table='vaccinations', mode=mode,properties=config)
